In [51]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [52]:
df = pd.read_csv("..//DataNettoyage//df_no_outliers.csv")
df['popularity'] = df['popularity'].astype(float)


In [53]:
df = df.drop(columns=['_id', 'artist_id', 'name'])

In [54]:
df['num_albums'] = df['albums'].apply(lambda x: len(eval(x)) if isinstance(x, str) else 0)
df = df.drop(columns=['albums'])


In [55]:
df = pd.get_dummies(df, columns=['genres'], drop_first=True)

In [56]:
# Définir les features (X) et la variable cible (y) pour la classification et la régression
df['is_popular'] = (df['popularity'] >= df['popularity'].median()).astype(int)
X_classification = df.drop(columns=['popularity', 'is_popular'])
y_classification = df['is_popular']

In [57]:
X_regression = df.drop(columns=['popularity'])
y_regression = df['popularity']

In [58]:
X_clustering = df.drop(columns=['popularity'])


In [59]:
from sklearn.model_selection import train_test_split

# Pour la classification
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_classification, y_classification, test_size=0.2, random_state=42)

# Pour la régression
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X_regression, y_regression, test_size=0.2, random_state=42)


## Classification : Prédire si un artiste est populaire

In [60]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib
from sklearn import tree
import graphviz

# Initialiser le modèle
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Entraîner le modèle
clf.fit(X_train_c, y_train_c)

# Prédire sur le test set
y_pred_c = clf.predict(X_test_c)

# Évaluer le modèle
accuracy = accuracy_score(y_test_c, y_pred_c)
print(f"Accuracy (Classification) : {accuracy:.2f}")

joblib.dump(clf, "classification_model.pkl")

estimator = clf.estimators_[0]

# Exporter l'arbre en format DOT
dot_data = tree.export_graphviz(
    estimator, out_file=None, feature_names=X_train_c.columns, 
    class_names=["Non Populaire", "Populaire"], filled=True, rounded=True, special_characters=True
)

# Convertir et enregistrer en PDF
graph = graphviz.Source(dot_data)
graph.render("decision_tree")


Accuracy (Classification) : 0.85


['classification_model.pkl']

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib

# Initialiser le modèle
clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Entraîner le modèle
clf.fit(X_train_c, y_train_c)

# Prédire sur le test set
y_pred_c = clf.predict(X_test_c)

# Évaluer le modèle
accuracy = accuracy_score(y_test_c, y_pred_c)
print(f"Accuracy (Classification) : {accuracy:.2f}")

joblib.dump(clf, "classification_model.pkl")


Accuracy (Classification) : 0.85


['classification_model.pkl']

## Régression : Prédire la popularité d’un artist

In [47]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import joblib


# Initialiser le modèle
reg = RandomForestRegressor(n_estimators=100, random_state=42)

# Entraîner le modèle
reg.fit(X_train_r, y_train_r)

# Prédire sur le test set
y_pred_r = reg.predict(X_test_r)

# Évaluer le modèle
mse = mean_squared_error(y_test_r, y_pred_r)
print(f"Mean Squared Error (Régression) : {mse:.2f}")

joblib.dump(reg, "regression_model.pkl")


Mean Squared Error (Régression) : 58.47


['regression_model.pkl']

In [48]:
from sklearn.cluster import KMeans
import joblib


# Définir le nombre de clusters
kmeans = KMeans(n_clusters=3, random_state=42)

# Entraîner le modèle
df['cluster'] = kmeans.fit_predict(X_clustering)

# Afficher les clusters attribués
print(df[['genres', 'cluster']].head(10))

joblib.dump(kmeans, "clustering_model.pkl")


KeyError: "['genres'] not in index"